In [ ]:
%%capture
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import timeit
import os

import delfi.distribution as dd
import delfi.inference as infer
import delfi.generator as dg

from delfi.simulator import TwoMoons
import delfi.summarystats as ds
from delfi.utils.viz import plot_pdf, probs2contours

from lfimodels.snl_exps.util import save_results, load_results
from snl.util.plot import plot_hist_marginals

fig_path = 'results/figs/'
fontsize=16

# panel (a)

In [ ]:
save_path = 'results/gauss_validationset/'

plot_seed = 42
exp_id = 'seed' + str(plot_seed)
log, tds, posteriors, _ = load_results(exp_id=exp_id, path=save_path)
pnl_a = plot_hist_marginals(tds[-1][0][-300:], lims=[-5,5])
pnl_a.set_figwidth(8)
pnl_a.set_figheight(8)
for ax in pnl_a.axes:
    ax.set_xticks([])
    ax.set_yticks([])
#    ax.axis('off')
    
pnl_a.axes[-5].set_xlabel(r'$\theta_1$', fontsize=fontsize)
pnl_a.axes[-4].set_xlabel(r'$\theta_2$', fontsize=fontsize)
pnl_a.axes[-3].set_xlabel(r'$\theta_3$', fontsize=fontsize)
pnl_a.axes[-2].set_xlabel(r'$\theta_4$', fontsize=fontsize)    
pnl_a.axes[-1].set_xlabel(r'$\theta_5$', fontsize=fontsize)         
    
PANEL_2A = fig_path +'fig2_a.svg'
plt.savefig(PANEL_2A, facecolor=plt.gcf().get_facecolor(), transparent=True)
    
pnl_a.show()

# panel (b)

In [ ]:
# load MMD results 

# panel (c)

In [ ]:
PANEL_2C = fig_path +'gauss_mmd_validationset_large.svg'

# assemble figure

In [ ]:
%run -i ./common.ipynb

In [ ]:
# FIGURE and GRID
FIG_HEIGHT_MM = 90
FIG_WIDTH_MM = 160  # set in NIPS2017 notebook to a default value for all figures

FIG_N_ROWS = 2
ROW_1_NCOLS = 2
ROW_1_HEIGHT_MM =      1.2 * (FIG_HEIGHT_MM / FIG_N_ROWS )
ROW_1_WIDTH_COL_1_MM =  1.5* (FIG_WIDTH_MM / ROW_1_NCOLS)

ROW_2_NCOLS = 2
ROW_2_HEIGHT_MM = 0.8 * FIG_HEIGHT_MM / FIG_N_ROWS
ROW_2_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_2_NCOLS
ROW_2_WIDTH_COL_2_MM = FIG_WIDTH_MM / ROW_2_NCOLS


fig = create_fig(FIG_WIDTH_MM, FIG_HEIGHT_MM)


fig = add_svg(fig, PANEL_2A, 
              -10, 
              0)
fig = add_label(fig, 
                'a)', 
                0, 
                8)
fig = add_svg(fig, PANEL_2C, 
              72,
              4) 
fig = add_label(fig, 
                'b)', 
                57, 
                8)



#fig = add_label(fig, 
#                'b, 
#                60, 
#                10)


if False:
    fig = add_grid(fig, 2, 2)
    fig = add_grid(fig, 160/3, 10, font_size_px=0.0001)


PATH_SVG = PATH_DROPBOX_FIGS + 'fig2.svg'
fig.save(PATH_SVG)
svg(PATH_SVG)
!$INKSCAPE --export-pdf $PATH_DROPBOX_FIGS/fig2.pdf $PATH_SVG


# NOISE-dims figure

somewhat confusing (historically grounded...) naming of experiments:
- 'v1' : d =  20 (m=12) aka 'this should kill SNL'
- 'v2' : d = 100 (m=92) aka 'this has to kill SNL'
- 'v3' : d =  60 (m=52) 
- 'v4' : d =  40 (m=32)

In [ ]:
# prep 
save_path = 'results/gauss_noisedims_v1/'
seeds = np.arange(43,62)

prior_mmds = np.zeros(len(seeds))
for i in range(len(seeds)):

    seed = seeds[i]
    exp_id = 'seed' + str(seed)
    prior_mmds[i] = np.load(os.path.join(save_path, exp_id, 'all_mmds_snl_N' + str(1000)+'.npy'))[0]
    
prior_mmd = np.mean(np.sqrt(prior_mmds))**2
    
posterior_mmd = 0.0009931369858651173 # computed from MMD of N=1000 against 5000 ground-truth samples
posterior_mmd = 0.0017898981617274767 # computed from MMD of N=1000 against 4000 *other* ground-truth samples

In [ ]:
save_path = 'results/gauss_noisedims_v1/'

seeds = np.arange(52,55)
sq_mmds_snpe_all = np.zeros((seeds.size, 40))
sq_mmds_snl_all = []
for i in range(len(seeds)):

    seed = seeds[i]
    exp_id = 'seed' + str(seed)
    sq_mmds_snpe = np.load(os.path.join(save_path, exp_id, 'all_mmds_N' + str(5000)+'.npy'))
    sq_mmds_snpe_all[i,:] = sq_mmds_snpe
    
    try:
        sq_mmds_snl = np.load(os.path.join(save_path, exp_id, 'all_mmds_snl_N' + str(1000)+'.npy'))
        sq_mmds_snl_all.append(sq_mmds_snl[1:])
    except:
        pass
    

pnl = plt.figure(figsize=(3,3))

for i in range(len(seeds)):
    plt.semilogx(1000*np.arange(1,sq_mmds_snl.size), np.sqrt(sq_mmds_snl_all)[i,:], 'rd-', linewidth=1.5)
    plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(sq_mmds_snpe_all[i]), 'kd-', linewidth=1.5)


plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(prior_mmd) *np.ones(sq_mmds_snpe.size), 
            'k--', linewidth=0.8)

plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(posterior_mmd) *np.ones(sq_mmds_snpe.size), 
            'k:', linewidth=0.8)


plt.legend(['SNL', 'APT'])
plt.ylabel('Maximum meand discrepancy')
plt.xlabel('Number of simulations (log scale)')
plt.ylim([0., 1.1])
plt.title('d = 20', fontsize=12)

PANEL_2nA = fig_path +'fig2n_a.svg'
plt.savefig(PANEL_2nA, facecolor=plt.gcf().get_facecolor(), transparent=True)

pnl.show()

In [ ]:
save_path = 'results/gauss_noisedims_v4/'

seeds = np.arange(52,55)
sq_mmds_snpe_all = np.zeros((seeds.size, 40))
sq_mmds_snl_all = []


pnl = plt.figure(figsize=(3,3))
for i in range(len(seeds)):

    seed = seeds[i]
    exp_id = 'seed' + str(seed)
    sq_mmds_snpe = np.load(os.path.join(save_path, exp_id, 'all_mmds_N' + str(1000)+'.npy'))
    sq_mmds_snpe_all[i,:] = sq_mmds_snpe
    
    try:
        sq_mmds_snl = np.load(os.path.join(save_path, exp_id, 'all_mmds_snl_N' + str(1000)+'.npy'))
        sq_mmds_snl_all.append(sq_mmds_snl[1:])
    except:
        pass


plt.semilogx(1000*np.arange(1,sq_mmds_snl.size), np.sqrt(sq_mmds_snl_all).T, 'rd-', linewidth=1.5,
        label='SNL')

plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(sq_mmds_snpe_all).T, 'kd-', linewidth=1.5,
        label='APT')

plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(prior_mmd) *np.ones(sq_mmds_snpe.size), 
            'k--', linewidth=0.8)

plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(posterior_mmd) *np.ones(sq_mmds_snpe.size), 
            'k:', linewidth=0.8)

#plt.legend()

plt.ylabel('Maximum meand discrepancy')
plt.xlabel('Number of simulations (log scale)')
plt.ylim([0., 1.1])
plt.title('d = 40', fontsize=12)

PANEL_2nB = fig_path +'fig2n_b.svg'
plt.savefig(PANEL_2nB, facecolor=plt.gcf().get_facecolor(), transparent=True)

pnl.show()

In [ ]:
save_path = 'results/gauss_noisedims_v3/'

seeds = np.arange(52,55)
sq_mmds_snpe_all = np.zeros((seeds.size, 40))
sq_mmds_snl_all = []
for i in range(len(seeds)):

    seed = seeds[i]
    exp_id = 'seed' + str(seed)
    sq_mmds_snpe = np.load(os.path.join(save_path, exp_id, 'all_mmds_N' + str(1000)+'.npy'))
    sq_mmds_snpe_all[i,:] = sq_mmds_snpe
    
    try:
        sq_mmds_snl = np.load(os.path.join(save_path, exp_id, 'all_mmds_snl_N' + str(1000)+'.npy'))
        sq_mmds_snl_all.append(sq_mmds_snl[1:])
    except:
        pass

pnl = plt.figure(figsize=(3,3))

plt.semilogx(1000*np.arange(1,sq_mmds_snl.size), np.sqrt(sq_mmds_snl_all).T, 'rd-', linewidth=1.5,
        label='SNL')

plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(sq_mmds_snpe_all).T, 'kd-', linewidth=1.5,
        label='APT')

plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(prior_mmd) *np.ones(sq_mmds_snpe.size), 
            'k--', linewidth=0.8)

plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(posterior_mmd) *np.ones(sq_mmds_snpe.size), 
            'k:', linewidth=0.8)

#plt.legend()
#plt.ylabel('Maximum meand discrepancy')
plt.xlabel('Number of simulations (log scale)')
plt.ylim([0., 1.1])
plt.title('d = 60', fontsize=12)

PANEL_2nC = fig_path +'fig2n_c.svg'
plt.savefig(PANEL_2nC, facecolor=plt.gcf().get_facecolor(), transparent=True)

pnl.show()

In [ ]:
save_path = 'results/gauss_noisedims_v2/'

seeds = np.arange(52,55)
sq_mmds_snpe_all = np.zeros((seeds.size, 40))
sq_mmds_snl_all = []
for i in range(len(seeds)):

    seed = seeds[i]
    exp_id = 'seed' + str(seed)
    sq_mmds_snpe = np.load(os.path.join(save_path, exp_id, 'all_mmds_N' + str(1000)+'.npy'))
    sq_mmds_snpe_all[i,:] = sq_mmds_snpe
    
    try:
        sq_mmds_snl = np.load(os.path.join(save_path, exp_id, 'all_mmds_snl_N' + str(1000)+'.npy'))
        sq_mmds_snl_all.append(sq_mmds_snl[1:])
    except:
        pass

pnl = plt.figure(figsize=(3,3))

plt.semilogx(1000*np.arange(1,sq_mmds_snl.size), np.sqrt(sq_mmds_snl_all).T, 'rd-', linewidth=1.5,
        label='SNL')

plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(sq_mmds_snpe_all).T, 'kd-', linewidth=1.5,
        label='APT')

plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(prior_mmd) *np.ones(sq_mmds_snpe.size), 
            'k--', linewidth=0.8)

plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(posterior_mmd) *np.ones(sq_mmds_snpe.size), 
            'k:', linewidth=0.8)

#plt.legend()
#plt.ylabel('Maximum meand discrepancy')
plt.xlabel('Number of simulations (log scale)')
plt.ylim([0.0, 1.1])
plt.title('d = 100', fontsize=12)

PANEL_2nD = fig_path +'fig2n_d.svg'
plt.savefig(PANEL_2nD, facecolor=plt.gcf().get_facecolor(), transparent=True)

pnl.show()

In [ ]:
%run -i ./common.ipynb

In [ ]:
# FIGURE and GRID
FIG_HEIGHT_MM = 38
FIG_WIDTH_MM = 160  # set in NIPS2017 notebook to a default value for all figures

FIG_N_ROWS = 2
ROW_1_NCOLS = 2
ROW_1_HEIGHT_MM =      1.2 * (FIG_HEIGHT_MM / FIG_N_ROWS )
ROW_1_WIDTH_COL_1_MM =  1.5* (FIG_WIDTH_MM / ROW_1_NCOLS)

ROW_2_NCOLS = 2
ROW_2_HEIGHT_MM = 0.8 * FIG_HEIGHT_MM / FIG_N_ROWS
ROW_2_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_2_NCOLS
ROW_2_WIDTH_COL_2_MM = FIG_WIDTH_MM / ROW_2_NCOLS


fig = create_fig(FIG_WIDTH_MM, FIG_HEIGHT_MM)


fig = add_svg(fig, PANEL_2nA, 
              3, 
              0)
fig = add_label(fig, 
                'a)', 
                0, 
                3)

fig = add_svg(fig, PANEL_2nB, 
              43, 
              0)
fig = add_label(fig, 
                'b)', 
                40, 
                3)

fig = add_svg(fig, PANEL_2nC, 
              83, 
              0)
fig = add_label(fig, 
                'c)', 
                80, 
                3)

fig = add_svg(fig, PANEL_2nD, 
              123,
              0) 
fig = add_label(fig, 
                'd)', 
                120, 
                3)



#fig = add_label(fig, 
#                'b, 
#                60, 
#                10)


if False:
    fig = add_grid(fig, 2, 2)
    fig = add_grid(fig, 160/3, 10, font_size_px=0.0001)


PATH_SVG = PATH_DROPBOX_FIGS + 'fig2n.svg'
fig.save(PATH_SVG)
svg(PATH_SVG)
!$INKSCAPE --export-pdf $PATH_DROPBOX_FIGS/fig2n.pdf $PATH_SVG


# mean +/- std's panel

In [ ]:
save_path = 'results/gauss_noisedims_v1/'

seeds = np.arange(52, 62)
sq_mmds_snpe_all = np.zeros((seeds.size, 40))
sq_mmds_snl_all = []
for i in range(len(seeds)):

    seed = seeds[i]
    exp_id = 'seed' + str(seed)
    sq_mmds_snpe = np.load(os.path.join(save_path, exp_id, 'all_mmds_N' + str(5000)+'.npy'))
    sq_mmds_snpe_all[i,:] = sq_mmds_snpe

    #plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(sq_mmds_snpe), 'kd-', linewidth=1.5)
    
    try:
        sq_mmds_snl = np.load(os.path.join(save_path, exp_id, 'all_mmds_snl_N' + str(1000)+'.npy'))
        #plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.sqrt(sq_mmds_snl), 'rd-', linewidth=1.5)
        sq_mmds_snl_all.append(sq_mmds_snl[1:])
    except:
        pass
    

plt.figure(figsize=(4,4))

plt.semilogx(1000*np.arange(1,sq_mmds_snl.size), np.mean(np.sqrt(sq_mmds_snl_all),axis=0), 'rd-', linewidth=1.5,
        label='SNL')

plt.fill_between(1000*np.arange(1,sq_mmds_snl.size),
                 np.mean(np.sqrt(sq_mmds_snl_all),axis=0) - np.std(np.sqrt(sq_mmds_snl_all),axis=0),
                 np.mean(np.sqrt(sq_mmds_snl_all),axis=0) + np.std(np.sqrt(sq_mmds_snl_all),axis=0),
                 color='r', alpha=0.2)

plt.semilogx(1000*np.arange(1,sq_mmds_snpe.size+1), np.mean(np.sqrt(sq_mmds_snpe_all),axis=0), 'kd-', linewidth=1.5,
        label='APT')

plt.fill_between(1000*np.arange(1,sq_mmds_snpe.size+1),
                 np.mean(np.sqrt(sq_mmds_snpe_all),axis=0) - np.std(np.sqrt(sq_mmds_snpe_all),axis=0),
                 np.mean(np.sqrt(sq_mmds_snpe_all),axis=0) + np.std(np.sqrt(sq_mmds_snpe_all),axis=0),
                 color='k', alpha=0.5)

plt.legend()
plt.ylabel('Maximum meand discrepancy')
plt.xlabel('Number of simulations (log scale)')

PANEL_2D = fig_path +'fig2_d.svg'
plt.savefig(PANEL_2D, facecolor=plt.gcf().get_facecolor(), transparent=True)

plt.show()

# Sanity-check Gallery

In [ ]:
from lfimodels.snl_exps.util import draw_sample_uniform_prior_33 as fast_sampler    
save_path = 'results/gauss_noisedims_v1//'

for plot_seed in range(43, 47):
    exp_id = 'seed' + str(plot_seed)
    log, tds, posteriors, _ = load_results(exp_id=exp_id, path=save_path)
    pnl_a = plot_hist_marginals(fast_sampler(posteriors[1], 1000), lims=[-5,5])
    pnl_a.set_figwidth(12)
    pnl_a.set_figheight(12)
    for ax in pnl_a.axes:
        ax.axis('off')

    pnl_a.show()

In [ ]:
import pickle 
import os

save_path = 'results/gauss_noisedims_v1/'

for plot_seed in range(43,62):
    
    print('seed ' + str(plot_seed))
    exp_id = 'seed' + str(plot_seed)
    
    try:
        file = os.path.join(save_path, exp_id, 'SNL_MAF')
        with open(file + '.pkl', 'rb') as f:
            learned_model = pickle.load(f)        

        file = os.path.join(save_path, exp_id, 'SNL_posteriors')
        with open(file + '.pkl', 'rb') as f:
            all_models = pickle.load(f)                   
    except:
        print('failed to load MAFs')
        
    xs = np.load(os.path.join(save_path, exp_id, 'xs.npy'))    
    ps = np.load(os.path.join(save_path, exp_id, 'ps.npy'))    
    
    pnl_a = plot_hist_marginals(ps[-1], lims=[-5,5])
    pnl_a.set_figwidth(12)
    pnl_a.set_figheight(12)
    for ax in pnl_a.axes:
        ax.axis('off')

    pnl_a.show()